[参考記事](https://qiita.com/koji_mats/items/0533fbdeb9012a7e1494#appendix-prefect%E3%82%92k8s%E3%81%B8%E3%83%87%E3%83%97%E3%83%AD%E3%82%A4%E3%81%99%E3%82%8B%E6%89%8B%E9%A0%86)

# メモ

* flowやtaskにはユニークな名前を付ける必要がある(そうでないと警告発生)

# 1. Hello World!!

In [ ]:
# ver1での書き方
# 2系で実施しているので、実行エラー
# from prefect import Flow, task

# @task
# def get_name():
#     return "world"

# @task
# def hello(who):
#     print("hello, {}!".format(who))

# with Flow("HelloWorld") as flow:
#     who = get_name()
#     hello_world = hello(who)

# flow.run()

In [3]:
# 記事の内容を2系で書き換え
from prefect import task, flow

@task(name="get name task")
def get_name():
    return("World")

@task(name="hello task")
def hello(who):
    print(f"Hello, {who}!")

@flow(name="greeting flow")
def greeting_flow():
    who = get_name()
    hello(who)

if __name__ == "__main__":
    greeting_flow()

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get name task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/690448971.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'hello task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/690448971.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'greeting flow' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/690448971.py:12' conflicts with a

09:05:44.848 | INFO    | prefect.engine - Created flow run 'tentacled-wildebeest' for flow 'greeting flow'

09:05:44.930 | INFO    | Flow run 'tentacled-wildebeest' - Created task run 'get name task-0' for task 'get name task'

09:05:44.931 | INFO    | Flow run 'tentacled-wildebeest' - Executing 'get name task-0' immediately...

09:05:44.980 | INFO    | Task run 'get name task-0' - Finished in state Completed()

09:05:44.995 | INFO    | Flow run 'tentacled-wildebeest' - Created task run 'hello task-0' for task 'hello task'

09:05:44.996 | INFO    | Flow run 'tentacled-wildebeest' - Executing 'hello task-0' immediately...

Hello, World!


09:05:45.076 | INFO    | Task run 'hello task-0' - Finished in state Completed()

09:05:45.092 | INFO    | Flow run 'tentacled-wildebeest' - Finished in state Completed('All states completed.')

# 2. パラメータ化

* flow実行時にパラメータを渡して、タスク内で使用する

In [4]:
# "name"という引数をflow側で用意して、taskに渡して実行する
from prefect import task, flow

@task(name="get name task")
def get_name(name):
    return name

@task(name="hello task")
def hello(who):
    print(f"Hello, {who}!")

@flow(name="greeting flow")
def greeting_flow(name):
    who = get_name(name)
    hello(who)

if __name__ == "__main__":
    greeting_flow("KOH")

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get name task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'hello task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'greeting flow' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:12' conflicts wit

09:23:58.012 | INFO    | prefect.engine - Created flow run 'hidden-salmon' for flow 'greeting flow'

09:23:58.083 | INFO    | Flow run 'hidden-salmon' - Created task run 'get name task-0' for task 'get name task'

09:23:58.084 | INFO    | Flow run 'hidden-salmon' - Executing 'get name task-0' immediately...

09:23:58.130 | INFO    | Task run 'get name task-0' - Finished in state Completed()

09:23:58.141 | INFO    | Flow run 'hidden-salmon' - Created task run 'hello task-0' for task 'hello task'

09:23:58.142 | INFO    | Flow run 'hidden-salmon' - Executing 'hello task-0' immediately...

Hello, KOH!


09:23:58.187 | INFO    | Task run 'hello task-0' - Finished in state Completed()

09:23:58.204 | INFO    | Flow run 'hidden-salmon' - Finished in state Completed('All states completed.')

# 3. 依存関係の定義

* task1 -> task2の順に実行したいようなケース

In [ ]:
# def my_flow():
#     x = task_1.submit()
#     # task 2 will wait for task_1 to complete
#     # y = task_2.submit(wait_for=[x])

In [1]:
from prefect import task, flow

@task(name="hello task")
def task1():
    print("Hello")


@task(name="world task")
def task2():
    print("World!!")

@flow(name="hello world flow")
def hello_world():
    x = task1.submit()
    y = task2.submit(wait_for=[x])

if __name__ == "__main__":
    hello_world()

10:36:40.015 | INFO    | prefect.engine - Created flow run 'brawny-unicorn' for flow 'hello world flow'

flowの実行完了


10:36:40.139 | INFO    | Flow run 'brawny-unicorn' - Created task run 'hello task-0' for task 'hello task'

10:36:40.140 | INFO    | Flow run 'brawny-unicorn' - Submitted task run 'hello task-0' for execution.

10:36:40.167 | INFO    | Flow run 'brawny-unicorn' - Created task run 'world task-0' for task 'world task'

...

Hello


10:36:40.211 | INFO    | Task run 'hello task-0' - Finished in state Completed()

World!!


10:36:40.248 | INFO    | Task run 'world task-0' - Finished in state Completed()

10:36:40.264 | INFO    | Flow run 'brawny-unicorn' - Finished in state Completed('All states completed.')

# 4. 動的Flow
* タスクのmapメソッドを使用して、実行時に上流タスクから渡された値に応じて動的にタスクを生成
  * mapの引数にリスト(を返すタスクの結果)を渡すと、```リストの各要素ごとにタスクを実行```
  * mapで実行されるタスクは可能であれば並列実行されるよう

In [23]:
from random import randint
from prefect import task, flow

@task
def random_list():
    return [i for i in range(randint(1, 4))]

@task
def squared(x):
    return x ** 2

@task
def sum_up(l):
    print(f"Squared results: {l}")
    print(f"Sum: {sum(l)}")


@flow(name="dynamic flow")
def dynamic_flow():
    l = random_list()
    squared_res = squared.map(l)
    sum_res = sum_up(squared_res)
    print(f"squared_res：{squared_res}")
    print(f"squared_resのタイプ：{type(squared_res)}")

if __name__ == "__main__":
    dynamic_flow()

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'random_list' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64112/2096989557.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'squared' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64112/2096989557.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'sum_up' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64112/2096989557.py:12' conflicts with another ta

11:04:54.482 | INFO    | prefect.engine - Created flow run 'loyal-stoat' for flow 'dynamic flow'

11:04:54.552 | INFO    | Flow run 'loyal-stoat' - Created task run 'random_list-0' for task 'random_list'

11:04:54.553 | INFO    | Flow run 'loyal-stoat' - Executing 'random_list-0' immediately...

11:04:54.602 | INFO    | Task run 'random_list-0' - Finished in state Completed()

...

11:04:54.876 | INFO    | Flow run 'loyal-stoat' - Executing 'sum_up-0' immediately...

Squared results: [0, 1, 4, 9]
Sum: 14


11:04:54.927 | INFO    | Task run 'sum_up-0' - Finished in state Completed()

squared_res：[PrefectFuture('squared-0'), PrefectFuture('squared-1'), PrefectFuture('squared-2'), PrefectFuture('squared-3')]
squared_resのタイプ：<class 'list'>


11:04:54.946 | INFO    | Flow run 'loyal-stoat' - Finished in state Completed('All states completed.')

# 5. 条件分岐
* 1系では、タスク（case、merge、ifelse、switch）は、prefect.tasks.control_flowからインポートして使用していた。
* 2系では、Pythonネイティブのif/else文を使って条件ロジックを指定することができます。

In [ ]:
# from random import randint
# from prefect import task, Flow
# from prefect.tasks.control_flow.conditional import ifelse

# @task
# def check_even():
#     return randint(0, 10) % 2 == 0

# @task
# def even():
#     print("It's Even!!")

# @task
# def odd():
#     print("It's Odd!!")

# with Flow("IfElseFlow") as flow:
#     cond = check_even()
#     result = ifelse(cond, even, odd)

# flow.run()


In [11]:
from random import randint
from prefect import task, flow

@task
def check_even():
    return randint(0, 10) % 2 == 0

@task
def even():
    print("It's Even!!")

@task
def odd():
    print("It's Odd!!")

@flow
def is_else_flow():
    cond = check_even.submit()
    print(f"cond: {cond}")
    print(f"cond.result(): {cond.result()}")
    if cond.result():
        even.submit(wait_for=[cond])
    else:
        odd.submit(wait_for=[cond])

if __name__ == "__main__":
    is_else_flow()

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'check_even' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/2061035519.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'even' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/2061035519.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'odd' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/2061035519.py:12' conflicts with another task. Con

11:34:52.680 | INFO    | prefect.engine - Created flow run 'prophetic-echidna' for flow 'is-else-flow'

cond: PrefectFuture('check_even-0')


11:34:52.755 | INFO    | Flow run 'prophetic-echidna' - Created task run 'check_even-0' for task 'check_even'

11:34:52.757 | INFO    | Flow run 'prophetic-echidna' - Submitted task run 'check_even-0' for execution.

...

11:34:52.823 | INFO    | Flow run 'prophetic-echidna' - Created task run 'even-0' for task 'even'

11:34:52.825 | INFO    | Flow run 'prophetic-echidna' - Submitted task run 'even-0' for execution.

It's Even!!


11:34:52.879 | INFO    | Task run 'even-0' - Finished in state Completed()

11:34:52.897 | INFO    | Flow run 'prophetic-echidna' - Finished in state Completed('All states completed.')

In [ ]:
from random import randint
from prefect import task, flow

@task
def check_even():
    return randint(0, 10) % 2 == 0

@task
def even():
    print("It's Even!!")

@task
def odd():
    print("It's Odd!!")

@flow
def is_else_flow():
    cond = check_even()
    if cond:
        even()
    else:
        odd()

if __name__ == "__main__":
    is_else_flow()

# 6. Flowの成功・失敗の条件
* 事前準備 -> 本処理 -> 後片付けのようなフローで、本処理の結果にかかわらず、後片付けをしたいケース

In [16]:
import random
from prefect import task, flow

@task
def do_something_important():
    bool_ = random.random() > 0.5
    print(f"Is the number > 0.5? {bool_}")
    if bool_:
        raise ValueError("Non-deterministic error has occured.")

@task
def fail():
    print("Failure")

@task
def succeed():
    print("Success")

@task
def always_run():
    print("Running regardless of upstream task's state")

@flow(log_prints=True)
def main_flow():
    a = do_something_important.submit()
    # equivalent of all_failed or any_failed trigger in Prefect 1.0:
    if a.wait() == "Failed":
        fail.submit()  # the task is skipped if the condition is not true
    # equivalent of all_succesful trigger in Prefect 1.0
    else:
        succeed.submit()  # the task is skipped on upstream failure

    # equivalent of always_run or all_finished trigger in Prefect 1.0:
    always_run.submit()

if __name__ == "__main__":
    main_flow()

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'do_something_important' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/537470895.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'fail' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/537470895.py:11' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'succeed' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_64897/537470895.py:15' conflicts with ano

11:52:16.250 | INFO    | prefect.engine - Created flow run 'rainbow-firefly' for flow 'main-flow'

11:52:16.324 | INFO    | Flow run 'rainbow-firefly' - Created task run 'do_something_important-0' for task 'do_something_important'

11:52:16.325 | INFO    | Flow run 'rainbow-firefly' - Submitted task run 'do_something_important-0' for execution.

11:52:16.364 | INFO    | Task run 'do_something_important-0' - Is the number > 0.5? True

...

11:52:16.488 | INFO    | Task run 'always_run-0' - Running regardless of upstream task's state

11:52:16.493 | INFO    | Task run 'succeed-0' - Finished in state Completed()

11:52:16.511 | INFO    | Task run 'always_run-0' - Finished in state Completed()

11:52:16.531 | ERROR   | Flow run 'rainbow-firefly' - Finished in state Failed('1/3 states failed.')

ValueError: Non-deterministic error has occured.